In [25]:
let splitOptions: StringSplitOptions = (StringSplitOptions.TrimEntries ||| StringSplitOptions.RemoveEmptyEntries);
let split (separators: char array) (x:string) = x.Split(separators, splitOptions)

let contains (input: string) (x:string) = x.Contains input

let numberStringToArray s = s |> (split([|' '|]) >> Seq.map int64)

type AlmanacMappingRange =
    { destinationStart: int64
      sourceStart: int64
      rangeLength: int64 }

let toAlmanacMappingRange (s: seq<int64>) = 
    let a = Array.ofSeq s
    { destinationStart = a.[0]; sourceStart = a.[1]; rangeLength = a.[2] }

let almanac (a: seq<int64>) (b: seq<AlmanacMappingRange>) =
    a
    |> Seq.map (fun a -> 
                    b 
                    // Determine correct range to apply the mapping
                    |> Seq.tryFind (fun mapping -> a >= mapping.sourceStart && a < (mapping.sourceStart + mapping.rangeLength))
                    // Apply the corresponding mapping or keep the initial seed value
                    |> function
                        | Some(x) -> x.destinationStart + (a - x.sourceStart)
                        | None -> a)



In [26]:
let seeds = File.ReadLines "./input" |> Seq.pick Some |> split([|':'|]) |> fun a -> a.[1] |> numberStringToArray

let findLowestLocation seeds =
    File.ReadLines "./input"
    |> Seq.skip 3
    |> String.concat "\n"
    |> split([|':'|])
    |> Seq.map (split [|'\n'|] >> Seq.filter (not << contains "map") >> Seq.map (numberStringToArray >> toAlmanacMappingRange))
    |> Seq.fold (almanac) seeds
    |> Seq.min

findLowestLocation seeds

388071289

In [27]:
// It is too slow to try all ranges with this input

// let seedsPart2 = seeds |> Seq.chunkBySize 2 |> Seq.map (fun range -> [range.[0]..(range.[0]+range.[1]-(int64 1))]) |> Seq.collect id
// findLowestLocation seedsPart2

// Let's apply the mapping in reverse order by starting at the lowest location possible until it matche one of our seed ranges.

let seedsRanges = seeds |> Seq.chunkBySize 2

let mappings = 
   File.ReadLines "./input"
   |> Seq.skip 3
   |> String.concat "\n"
   |> split([|':'|])
   |> Seq.map (split [|'\n'|] >> Seq.filter (not << contains "map") >> Seq.map (numberStringToArray >> toAlmanacMappingRange))

let reverseAlmanac (a: int64) (b: seq<AlmanacMappingRange>) =
   b 
   // Determine correct range to apply the mapping
   |> Seq.tryFind (fun mapping -> a >= mapping.destinationStart && a < (mapping.destinationStart + mapping.rangeLength))
   // Apply the corresponding mapping or keep the initial seed value
   |> function
      | Some(x) -> x.sourceStart + (a - x.destinationStart)
      | None -> a

let seedValueFromLocation location = 
   mappings
   |> Seq.rev
   |> Seq.fold (reverseAlmanac) location

let seedValueValid value =
   seedsRanges
   |> Seq.tryFind (fun range -> value >= range.[0] && value < (range.[0] + range.[1]))
   |> Option.isSome
   

// This is still too slow... but it worked
let mutable index = 0L
while not (index |> seedValueFromLocation |> seedValueValid) do
   if (index % 1_000_000L = 0) then printfn "%d" index
   index <- index + 1L

printfn "%d" index


84206669
